In [2]:
import tensorflow as tf
import tensorflow_datasets as tfds

raw_train_set, raw_valid_set, raw_test_set = tfds.load(
    name = "imdb_reviews",
    split = ["train[:90%]", "train[90%:]", "test"],
    as_supervised = True
)

tf.random.set_seed(42)

train_set = raw_train_set.shuffle(5000, seed=42).batch(32).prefetch(1)
valid_set = raw_valid_set.batch(32).prefetch(1)
test_set = raw_test_set.batch(32).prefetch(1)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling C:\Users\User\tensorflow_datasets\imdb_reviews\plain_text\1.0.0.incompleteRUFL8B\imdb_reviews-train.…

Generating test examples...: 0 examples [00:00, ? examples/s]

Shuffling C:\Users\User\tensorflow_datasets\imdb_reviews\plain_text\1.0.0.incompleteRUFL8B\imdb_reviews-test.t…

Generating unsupervised examples...: 0 examples [00:00, ? examples/s]

Shuffling C:\Users\User\tensorflow_datasets\imdb_reviews\plain_text\1.0.0.incompleteRUFL8B\imdb_reviews-unsupe…

Dataset imdb_reviews downloaded and prepared to C:\Users\User\tensorflow_datasets\imdb_reviews\plain_text\1.0.0. Subsequent calls will reuse this data.


In [5]:
for review, label in raw_train_set.take(4):
    print(review.numpy().decode("utf-8"))
    print("Label:", label.numpy())
    print("-----------------------")

This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.
Label: 0
-----------------------
I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell asleep because the film was rubbi

In [9]:
vocab_size = 1000
text_vec_layer = tf.keras.layers.TextVectorization(max_tokens=vocab_size)
text_vec_layer.adapt(train_set.map(lambda reviews, labels:reviews))

In [11]:
embed_size = 128
tf.random.set_seed(42)

model = tf.keras.Sequential([
    text_vec_layer,
    tf.keras.layers.Embedding(vocab_size, embed_size),
    tf.keras.layers.GRU(128),
    tf.keras.layers.Dense(1, activation="sigmoid")
])

model.compile(
    loss = tf.keras.losses.binary_crossentropy,
    optimizer = tf.keras.optimizers.Nadam(),
    metrics = ["accuracy"]
)

history = model.fit(train_set, validation_data=valid_set, epochs=2)

Epoch 1/2
704/704 [==============================] - 519s 732ms/step - loss: 0.6935 - accuracy: 0.4969 - val_loss: 0.6930 - val_accuracy: 0.5016
Epoch 2/2
704/704 [==============================] - 515s 732ms/step - loss: 0.6928 - accuracy: 0.5041 - val_loss: 0.6938 - val_accuracy: 0.5004


#### Masking

In [17]:
embed_size = 128
tf.random.set_seed(42)

model = tf.keras.Sequential([
    text_vec_layer,
    tf.keras.layers.Embedding(vocab_size, embed_size, mask_zero=True),
    tf.keras.layers.GRU(128),
    tf.keras.layers.Dense(1, activation="sigmoid")
])

model.compile(
    loss = tf.keras.losses.binary_crossentropy,
    optimizer = tf.keras.optimizers.Nadam(),
    metrics = ["accuracy"]
)

history = model.fit(train_set, validation_data=valid_set, epochs=2)

Epoch 1/2
704/704 [==============================] - 573s 806ms/step - loss: 0.5017 - accuracy: 0.7591 - val_loss: 0.4031 - val_accuracy: 0.8264
Epoch 2/2
704/704 [==============================] - 550s 782ms/step - loss: 0.3557 - accuracy: 0.8488 - val_loss: 0.3487 - val_accuracy: 0.8468


####  Manual Masking

In [18]:
inputs = tf.keras.layers.Input(shape=[], dtype=tf.string)
token_ids = text_vec_layer(inputs)

mask = tf.math.not_equal(token_ids, 0)
Z = tf.keras.layers.Embedding(vocab_size, embed_size)(token_ids)
Z = tf.keras.layers.GRU(128, dropout=0.2)(Z, mask=mask)

outputs = tf.keras.layers.Dense(1, activation="sigmoid")(Z)
model = tf.keras.Model(inputs=[inputs], outputs=[outputs])

In [19]:
model.compile(
    loss = tf.keras.losses.binary_crossentropy,
    optimizer = tf.keras.optimizers.Nadam(),
    metrics = ["accuracy"]
)

history = model.fit(train_set, validation_data=valid_set, epochs=2)

Epoch 1/2
704/704 [==============================] - 577s 813ms/step - loss: 0.5826 - accuracy: 0.6889 - val_loss: 0.4565 - val_accuracy: 0.8044
Epoch 2/2
704/704 [==============================] - 588s 836ms/step - loss: 0.4161 - accuracy: 0.8086 - val_loss: 0.3606 - val_accuracy: 0.8496


#### Using Ragged Tensors

In [20]:
text_vec_layer_ragged = tf.keras.layers.TextVectorization(max_tokens=vocab_size, ragged=True)
text_vec_layer_ragged.adapt(train_set.map(lambda reviews, labels:reviews))

#### Comapre: Ragged Tensor Representation With Regular Tensor Representation

In [21]:
text_vec_layer_ragged(["Greate movie!", "This is DiCaprio's best role"])

<tf.RaggedTensor [[1, 18], [11, 7, 1, 116, 217]]>

In [22]:
text_vec_layer(["Greate movie!", "This is DiCaprio's best role"])

<tf.Tensor: shape=(2, 5), dtype=int64, numpy=
array([[  1,  18,   0,   0,   0],
       [ 11,   7,   1, 116, 217]], dtype=int64)>

In [23]:
embed_size = 128
tf.random.set_seed(42)
model = tf.keras.Sequential([
    text_vec_layer_ragged,
    tf.keras.layers.Embedding(vocab_size, embed_size),
    tf.keras.layers.GRU(128),
    tf.keras.layers.Dense(1, activation="sigmoid")
])

model.compile(
    loss = tf.keras.losses.binary_crossentropy,
    optimizer = tf.keras.optimizers.Nadam(),
    metrics = ["accuracy"]
)

history = model.fit(train_set, validation_data=valid_set, epochs=2)

Epoch 1/2
704/704 [==============================] - 543s 766ms/step - loss: 0.4966 - accuracy: 0.7495 - val_loss: 0.3672 - val_accuracy: 0.8456
Epoch 2/2
704/704 [==============================] - 530s 752ms/step - loss: 0.3301 - accuracy: 0.8594 - val_loss: 0.3082 - val_accuracy: 0.8736


#### Reusing Pretrained Embeddings and Language Models

In [24]:
import os
import tensorflow_hub as hub

os.environ["TFHUB_CACHE_DIR"] = "my_tfhub_cache"
tf.random.set_seed(42)

model = tf.keras.Sequential([
    hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder/4", trainable=True, dtype=tf.string, input_shape=[]),
    tf.keras.layers.Dense(64, activation="relu"),
    tf.keras.layers.Dense(1, activation="sigmoid")
])

model.compile(
    loss = tf.keras.losses.binary_crossentropy,
    optimizer = tf.keras.optimizers.Nadam(),
    metrics = ["accuracy"]
)

history = model.fit(train_set, validation_data=valid_set, epochs=2)

Epoch 1/2
704/704 [==============================] - 2635s 4s/step - loss: 0.2935 - accuracy: 0.8764 - val_loss: 0.2310 - val_accuracy: 0.9040
Epoch 2/2
704/704 [==============================] - 2440s 3s/step - loss: 0.0251 - accuracy: 0.9929 - val_loss: 0.3330 - val_accuracy: 0.8952
